# Introduction
This kernel accompanies the [blog post](https://lavanya.ai/searching-for-dark-matter/) I wrote about searching for traces of dark matter in data produced by CERN.

I’ve recently been having a lot of fun playing with the Large Hadron Collider datasets, and I thought I’ll share some of the things I’ve learnt along the way. The 2 main things I’ll be exploring in this post are:

- **Particle identification**: training a classifier to detect electrons, protons, muons, kaons and pions
- **Searching for dark matter**: training a classifier to distinguish between background noise and the signal, and then applying clustering algorithms to find potential traces of dark matter in this signal

You can get more background by reading the blog post here: https://lavanya.ai/searching-for-dark-matter/

PS: I would love to hear what you think about the post and the kernel, and any suggestions you have for how I can improve it! This kernel is a WIP. I'll add to it over the next few weeks as I explore the dataset further. Thank you Coursera for presenting some of the ideas that inspired this kernel!

## If you like this kernel, please give it an upvote. Thank you! :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import xgboost as xg
import tables as tb
from tqdm import tqdm
from itertools import cycle, islice
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import make_blobs
from IPython import display
from sklearn.neighbors import BallTree, KDTree, DistanceMetric
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import Normalizer
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import StratifiedKFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import log_loss
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
import utils

%matplotlib inline
%pylab inline

# Make the images larger
plt.rcParams['figure.figsize'] = (16, 9)
figsize = (10,10)
point_size=150
point_border=0.8

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

# Particle Identification

In [ ]:
training = pd.read_csv('../input/particle-identification/training.csv')
test = pd.read_csv('../input/particle-identification/test.csv')

In [ ]:
training.shape

In [ ]:
test.shape

In [ ]:
training.head()

In [ ]:
test.head()

From the dataset description, we know that the features describe particle responses in the detector systems, and represent the following:

- ID - id value for tracks (presents only in the test file for the submitting purposes)
- Label - string valued observable denoting particle types. Can take values "Electron", "Muon", "Kaon", "Proton", "Pion" and "Ghost". This column is absent in the test file.
- FlagSpd - flag (0 or 1), if reconstructed track passes through Spd
- FlagPrs - flag (0 or 1), if reconstructed track passes through Prs
- FlagBrem - flag (0 or 1), if reconstructed track passes through Brem
- FlagEcal - flag (0 or 1), if reconstructed track passes through Ecal
- FlagHcal - flag (0 or 1), if reconstructed track passes through Hcal
- FlagRICH1 - flag (0 or 1), if reconstructed track passes through the first RICH detector
- FlagRICH2 - flag (0 or 1), if reconstructed track passes through the second RICH detector
- FlagMuon - flag (0 or 1), if reconstructed track passes through muon stations (Muon)
- SpdE - energy deposit associated to the track in the Spd
- PrsE - energy deposit associated to the track in the Prs
- EcalE - energy deposit associated to the track in the Hcal
- HcalE - energy deposit associated to the track in the Hcal
- PrsDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Prs
- BremDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Brem
- TrackP - particle momentum
- TrackPt - particle transverse momentum
- TrackNDoFSubdetector1  - number of degrees of freedom for track fit using hits in the tracking sub-detector1
- TrackQualitySubdetector1 - chi2 quality of the track fit using hits in the tracking sub-detector1
- TrackNDoFSubdetector2 - number of degrees of freedom for track fit using hits in the tracking sub-detector2
- TrackQualitySubdetector2 - chi2 quality of the track fit using hits in the  tracking sub-detector2
- TrackNDoF - number of degrees of freedom for track fit using hits in all tracking sub-detectors
- TrackQualityPerNDoF - chi2 quality of the track fit per degree of freedom
- TrackDistanceToZ - distance between track and z-axis (beam axis)
- Calo2dFitQuality - quality of the 2d fit of the clusters in the calorimeter 
- Calo3dFitQuality - quality of the 3d fit in the calorimeter with assumption that particle was electron
- EcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Ecal
- EcalDLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from Ecal
- EcalShowerLongitudinalParameter - longitudinal parameter of Ecal shower
- HcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Hcal
- HcalDLLbeMuon - delta log-likelihood for a particle candidate to be using information from Hcal
- RICHpFlagElectron - flag (0 or 1) if momentum is greater than threshold for electrons to produce Cherenkov light
- RICHpFlagProton - flag (0 or 1) if momentum is greater than threshold for protons to produce Cherenkov light
- RICHpFlagPion - flag (0 or 1) if momentum is greater than threshold for pions to produce Cherenkov light
- RICHpFlagKaon - flag (0 or 1) if momentum is greater than threshold for kaons to produce Cherenkov light
- RICHpFlagMuon - flag (0 or 1) if momentum is greater than threshold for muons to produce Cherenkov light
- RICH_DLLbeBCK  - delta log-likelihood for a particle candidate to be background using information from RICH
- RICH_DLLbeKaon - delta log-likelihood for a particle candidate to be kaon using information from RICH
- RICH_DLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from RICH
- RICH_DLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from RICH
- RICH_DLLbeProton - delta log-likelihood for a particle candidate to be proton using information from RICH
- MuonFlag - muon flag (is this track muon) which is determined from muon stations
- MuonLooseFlag muon flag (is this track muon) which is determined from muon stations using looser criteria
- MuonLLbeBCK - log-likelihood for a particle candidate to be not muon using information from muon stations
- MuonLLbeMuon - log-likelihood for a particle candidate to be muon using information from muon stations
- DLLelectron - delta log-likelihood for a particle candidate to be electron using information from all subdetectors
- DLLmuon - delta log-likelihood for a particle candidate to be muon using information from all subdetectors
- DLLkaon - delta log-likelihood for a particle candidate to be kaon using information from all subdetectors
- DLLproton - delta log-likelihood for a particle candidate to be proton using information from all subdetectors
- GhostProbability - probability for a particle candidate to be ghost track. This variable is an output of classification model used in the tracking algorithm.

Spd stands for Scintillating Pad Detector, Prs - Preshower, Ecal - electromagnetic calorimeter, Hcal - hadronic calorimeter, Brem denotes traces of the particles that were deflected by detector.

In [ ]:
def my_percentile(arr, w, q):
    left = 0.
    right = (w).sum()
    sort_inds = np.argsort(arr, axis=0)
    if left/right >= q/100.:
        return arr[0]
    for i in sort_inds:
        left += w[i]
        if left/right >= q/100.:
            return arr[i]

def plot(prediction, spectator, cut, percentile=True, weights=None, n_bins=100,
              color='b', marker='o', ms=4, label="MVA", fmt='o', markeredgecolor='b', markeredgewidth=2, ecolor='b'):
    if weights is None:
        weights = np.ones(len(prediction))

    if percentile:
        if weights is None:
            cut = np.percentile(prediction, 100-cut)
        else:
            cut = my_percentile(prediction, weights, 100-cut)
    
    edges = np.linspace(spectator.min(), spectator.max(), n_bins)
    
    xx = []
    yy = []
    xx_err = []
    yy_err = []
    
    for i_edge in range(len(edges)-1):

        left = edges[i_edge]
        right = edges[i_edge + 1]
        
        N_tot_bin = weights[((spectator >= left) * (spectator < right))].sum()
        N_cut_bin = weights[((spectator >= left) * (spectator < right) * (prediction >= cut))].sum()
        
        if N_tot_bin != 0:
            
            x = 0.5 * (right + left)
            y = 1. * N_cut_bin / N_tot_bin
            
            if y > 1.:
                y = 1.
            if y < 0:
                y = 0
            
            xx.append(x)
            yy.append(y)
            
            x_err = 0.5 * (right - left)
            y_err = np.sqrt(y*(1-y)/N_tot_bin)
            
            xx_err.append(x_err)
            yy_err.append(y_err)
        
        else:
            pass

    plt.errorbar(xx, yy, yerr=yy_err, xerr=xx_err, fmt=fmt, color=color, marker=marker, ms=ms, label=label, markeredgecolor=markeredgecolor, markeredgewidth=markeredgewidth, ecolor=ecolor)
    
    return cut

## Explore the dataset

In [ ]:
training['Label'].value_counts()

## Train an AdaBoost model 

In [ ]:
# Get numeric labels for each of the string labels, to make them compatible with our model
label_class_correspondence = {'Electron': 0, 'Ghost': 1, 'Kaon': 2, 'Muon': 3, 'Pion': 4, 'Proton': 5}
class_label_correspondence = {0: 'Electron', 1: 'Ghost', 2: 'Kaon', 3: 'Muon', 4: 'Pion', 5: 'Proton'}

def get_class_ids(labels):
    return np.array([label_class_correspondence[alabel] for alabel in labels])

In [ ]:
training['Class'] = get_class_ids(training.Label.values)

Since we're trying to predict the label (or class), we remove them from our list of features.

In [ ]:
features = list(set(training.columns) - {'Label', 'Class'})

Split the training data into training and validation sets.

In [ ]:
train, valid = train_test_split(training, random_state=42, train_size=0.90, test_size=0.10)
print(train.shape[0], valid.shape[0])

In [ ]:
train.head()

In [ ]:
clf = AdaBoostClassifier(n_estimators=100, learning_rate=0.01, random_state=42,
                             base_estimator=DecisionTreeClassifier(max_depth=6, min_samples_leaf=30, random_state=42))
clf.fit(train[features].values, train.Class.values)

In [ ]:
proba_clf = clf.predict_proba(valid[features].values)
log_loss(valid.Class.values, proba_clf)

In [ ]:
def roc_curves(predictions, labels):
    plt.figure(figsize=(9, 6))
    u_labels = np.unique(labels)
    for lab in u_labels:
        y_true = labels == lab
        y_pred = predictions[:, lab]
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        auc = roc_auc_score(y_true, y_pred)
        plt.plot(tpr, 1-fpr, linewidth=3, label=class_label_correspondence[lab] + ', AUC = ' + str(np.round(auc, 4)))
        plt.xlabel('Signal efficiency (TPR)', size=15)
        plt.ylabel("Background rejection (1 - FPR)", size=15)
        plt.xticks(size=15)
        plt.yticks(size=15)
        plt.xlim(0., 1)
        plt.ylim(0., 1)
        plt.legend(loc='lower left', fontsize=15)
        plt.title('One particle vs rest ROC curves', loc='right', size=15)
        plt.grid(b=1)

In [ ]:
roc_curves(proba_clf, valid.Class.values)

In [ ]:
def efficiency(predictions, labels, spectator, eff=60, n_bins=20, xlabel='Spectator'):
    plt.figure(figsize=(5.5*2, 3.5*3))
    u_labels = np.unique(labels)
    for lab in u_labels:
        y_true = labels == lab
        pred = predictions[y_true, lab]
        spec = spectator[y_true]
        plt.subplot(3, 2, lab+1)
        plot(pred, spec, cut=eff, percentile=True, weights=None, n_bins=n_bins, color='1', marker='o', 
                  ms=7, label=class_label_correspondence[lab], fmt='o')
        
        plt.plot([spec.min(), spec.max()], [eff / 100., eff / 100.], label='Global signal efficiecny', color='r', linewidth=3)
        plt.legend(loc='best', fontsize=12)
        plt.xticks(size=12)
        plt.yticks(size=12)
        plt.ylabel('Signal efficiency (TPR)', size=12)
        plt.xlabel(xlabel,size=12)
        plt.ylim(0, 1)
        plt.xlim(spec.min(), spec.max())
        plt.grid(b=1)
    plt.tight_layout()
        

def efficiency_on_p(predictions, labels, spectator, eff=60, n_bins=20):
    sel = spectator < 200 * 10**3
    efficiency(predictions[sel], labels[sel], spectator[sel] / 10**3, eff, n_bins, 'Momentum, GeV/c')

In [ ]:
efficiency_on_p(proba_clf, valid.Class.values, valid.TrackP.values, 60, 50)
plt.show()

In [ ]:
def efficiency_on_pt(predictions, labels, spectator, eff=60, n_bins=20):
    sel = spectator < 10 * 10**3
    efficiency(predictions[sel], labels[sel], spectator[sel] / 10**3, eff, n_bins, 'Transverse momentum, GeV/c')

In [ ]:
efficiency_on_pt(proba_clf, valid.Class.values, valid.TrackPt.values, 60, 50)
plt.show()

## Train a Neural Network

Training a second, neural network model and comparing performance via ROC curves

In [ ]:
def nn_model(input_dim):
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim))
    model.add(Activation('tanh'))

    model.add(Dense(6))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    return model

In [ ]:
nn = nn_model(len(features))
nn.fit(train[features].values, np_utils.to_categorical(train.Class.values), verbose=1, nb_epoch=5, batch_size=256)

In [ ]:
proba_nn = nn.predict_proba(valid[features].values)
log_loss(valid.Class.values, proba_nn)

In [ ]:
proba_nn

In [ ]:
roc_curves(proba_nn, valid.Class.values)

Just to compare performances, here are the ROC curves for our AdaBoostClassifier model.

In [ ]:
roc_curves(proba_clf, valid.Class.values)

As we can see, AdaBoostClassifier performs slightly better than the neural net across the board for all particles.

In [ ]:
# release unreferenced memory to ensure we don't run out of memory
import gc
gc.collect()

In [ ]:
del train, test, valid, training

# Searching for dark matter

In [ ]:
train = pd.read_hdf('../input/dark-matter-from-opera-experiments/open30.h5') # pick just a single brick
test = pd.read_hdf('../input/dark-matter-from-opera-experiments/test.h5')

In [ ]:
train.shape

In [ ]:
test = test.reset_index(drop=True)
test.shape

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
train.signal.value_counts()

In [ ]:
test.head()

Create a dataframe of signals for clustering later on

In [ ]:
train_signal = train.copy()
train_signal.head()

In [ ]:
train_signal = train_signal[train['signal']==1]
train_signal.signal.value_counts()

In [ ]:
train_signal.head()

## Explore dataset and find neighboring base tracks

Plot showers in the brick

In [ ]:
CMAP = sns.diverging_palette(220, 20, s=99, as_cmap=True, n=2500)

def plot3D(X, target, elev=0, azim=0, title=None, sub=111):
    x = X[:, 0]
    y = X[:, 1]
    z = X[:, 2]
    
    fig = plt.figure(figsize=(12, 8))
    ax = Axes3D(fig)
    mappab = ax.scatter(x, y, z, c=target, cmap=CMAP)

    if title is not None:
        ax.set_title(title)
    ax.set_xlabel('Component 1')
    ax.set_ylabel('Component 2')
    ax.set_zlabel('Component 3')

    ax.view_init(elev=elev, azim=azim)
    fig.colorbar(mappable=mappab, label='Target variable')
    plt.show()
    
feat_XY = ['TX', 'TY', 'X', 'Y']
first = train.loc[train.data_ind == 21, :]
plot3D(first.loc[first.signal==1, ['Z', 'X', 'Y']].values,
       first.loc[first.signal==1].signal.values, elev=20, azim=60)

In [ ]:
plot3D(first.loc[first.signal==1, ['Z', 'X', 'Y']].values,
       first.loc[first.signal==1].signal.values, elev=45, azim=0)

Closer look at base track distribution along the axes

In [ ]:
axis = 'X'

fig = plt.figure(figsize = [20, 10])
fig.add_subplot(221)
plt.hist(first.loc[first.signal == 1, axis], bins=500, histtype='step')
fig.add_subplot(222)
plt.hist(first.loc[first.signal == 0, axis], bins=500, histtype='step')
fig.add_subplot(223)
values_X = plt.hist(first.loc[:, axis], bins=500, histtype='step')
plt.show()

In [ ]:
axis = 'Y'

fig = plt.figure(figsize = [20, 10])
fig.add_subplot(221)
plt.hist(first.loc[first.signal == 1, axis], bins=500, histtype='step')
fig.add_subplot(222)
plt.hist(first.loc[first.signal == 0, axis], bins=500, histtype='step')
fig.add_subplot(223)
values_X = plt.hist(first.loc[:, axis], bins=500, histtype='step')
plt.show()

In [ ]:
axis = 'Z'

fig = plt.figure(figsize = [20, 10])
fig.add_subplot(221)
plt.hist(first.loc[first.signal == 1, axis], bins=500, histtype='step')
fig.add_subplot(222)
plt.hist(first.loc[first.signal == 0, axis], bins=500, histtype='step')
fig.add_subplot(223)
values_X = plt.hist(first.loc[:, axis], bins=500, histtype='step')
plt.show()

Group base tracks from neighboring plates (see blog post for intuition behind why we do this)

In [ ]:
def add_neighbours(df, k, metric='minkowski'):
    res = []
    
    for data_ind in tqdm(np.unique(df.data_ind)):
        ind = df.loc[df.data_ind == data_ind].copy()
        ind[['TX', 'TY']] *= 1293
        values = np.unique(ind.Z)
        
        for j in range(1, len(values)):
            z, z_next = (ind.loc[ind.Z == values[j-1]].copy(),
                         ind.loc[ind.Z == values[j]].copy())
            
            b_tree = BallTree(z_next[feat_XY], metric=metric)
            d, i = b_tree.query(z[feat_XY], k=min(k, len(z_next)))
            
            for m in range(i.shape[1]):
                data = z_next.iloc[i[:, m]]
                z_copy = z.copy()
                for col in feat_XY + ['Z']:
                    z_copy[col + '_pair'] = data[col].values
                res.append(z_copy)
            
        res.append(z_next)
        
    res = pd.concat(res)
    for col in feat_XY + ['Z']:
        res['d' + col] = res[col].values - res[col + '_pair'].values
    return res

def balance_train(df, k):
    data = add_neighbours(df, k=k)
    noise = data.event_id == -999
    signal, not_signal = data.loc[np.logical_not(noise)], data.loc[noise]
    noise_part = not_signal.sample(len(signal))
    return pd.concat([signal, noise_part]).reset_index(drop=True)
train = []
for file in glob.glob('../input/dark-matter-from-opera-experiments/open*.h5')[:5]: # just 5 bricks
    train.append(balance_train(pd.read_hdf(file), k=3))
train = pd.concat(train)

## Using a Neural Net to separate signal from background

In [ ]:
y_train = train.signal
X_train = train.drop(['event_id', 'signal', 'data_ind'], axis=1)

In [ ]:
transformer = Normalizer()
X_train_norm = transformer.fit_transform(X_train.fillna(0))

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_index, val_index = next(sss.split(X_train_norm, y_train))

In [ ]:
def nn_model(input_dim):
    model = Sequential()
    model.add(Dense(256, input_dim=input_dim))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam())
    return model

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto'),
            ModelCheckpoint('{val_loss:.4f}.hdf5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')]

In [ ]:
nn = nn_model(X_train_norm.shape[1])
nn.fit(X_train_norm, y_train, validation_split=0.2, epochs=20, verbose=2, batch_size=256, shuffle=True, callbacks=callbacks)

In [ ]:
prepared_test = add_neighbours(test, k=3)
X_test = prepared_test.drop(['data_ind'], axis=1)

In [ ]:
X_test_norm = transformer.transform(X_test.fillna(0))
X_test = transformer.transform(X_test.fillna(0))

In [ ]:
X_test_norm[:5]

In [ ]:
probas = nn.predict_proba(X_test_norm)

In [ ]:
probas = np.squeeze(probas)

In [ ]:
df = pd.DataFrame({'id': prepared_test.index, 'signal': probas}).groupby('id')
agg = df.aggregate(('mean')).loc[:, ['signal']]

In [ ]:
agg.head()

## Using KMeans to find signal clusters

In [ ]:
train_signal.fillna(0, inplace=True)

In [ ]:
'''
eps=0.000001
min_samples=2
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
clustering_labels = dbscan.fit_predict(train_signal)
'''

**Find the optimal number of clusters**

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(train_signal)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]
k = [inertias.index(i) for i in inertias]

In [ ]:
plt.plot(k, inertias, linewidth=2.0)
line, = plt.plot(k, inertias, 'o')
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)

The inertia's rate of decline flattens around k=6 clusters. So we'll train a KMeans with 6 clusters.

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42).fit(train_signal)
clustering_labels = kmeans.labels_

In [ ]:
train_signal.shape

In [ ]:
clustering_labels.shape

In [ ]:
clusters = train_signal
clusters['cluster'] = clustering_labels

In [ ]:
X_sample = train_signal.sample(frac=0.1, random_state=42)

In [ ]:
X_sample.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_sample.X, X_sample.Y, X_sample.Z, c=X_sample.cluster)

## If you like this kernel, please give it an upvote. Thank you! :)